In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('..\\Data\\raw\\twcs\\twcs.csv')

In [3]:
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [4]:
data["in_response_to_tweet_id"] = data["in_response_to_tweet_id"].fillna(-1).astype(int)
data["response_tweet_id"] = data["response_tweet_id"].fillna(-1)

In [5]:
data

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,-1,1
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6
...,...,...,...,...,...,...,...
2811769,2987947,sprintcare,False,Wed Nov 22 08:43:51 +0000 2017,"@823869 Hey, we'd be happy to look into this f...",-1,2987948
2811770,2987948,823869,True,Wed Nov 22 08:35:16 +0000 2017,@115714 wtf!? I’ve been having really shitty s...,2987947,-1
2811771,2812240,121673,True,Thu Nov 23 04:13:07 +0000 2017,@143549 @sprintcare You have to go to https://...,-1,2812239
2811772,2987949,AldiUK,False,Wed Nov 22 08:31:24 +0000 2017,"@823870 Sounds delicious, Sarah! 😋 https://t.c...",-1,2987950


In [6]:
unique_users = data[data["inbound"] == True]["author_id"].unique()

unique_users = unique_users[:10000] 

In [7]:
def extract_responses(response_ids):
    try:
        return [int(response_ids)]
    except:
        return [int(x) for x in response_ids.split(",")]

def extract_conversation(conv,response_num,data,comp_name):
    if response_num == -1:
        
        
        #print(conv)
        #print("CONVERSATION ENDED\n\n\n\n")
        
        return conv,comp_name
    
    else:
        
        row = data[data["tweet_id"] == response_num]
        if len(row) ==0:
            #print(conv)
            #print("CONVERSATION ENDED\n\n\n\n")
            return conv,comp_name
        
        
        conv += "\n"
        inbound_val = row["inbound"].values[0]
        if inbound_val:
            conv += "Customer: "
        else:
            conv += "Company: "
            if comp_name == None:
                comp_name = row["author_id"].values[0]
        conv += row["text"].values[0]
        
        responses = extract_responses(row["response_tweet_id"].values[0])
        for i in responses:
            
            conv,comp_name = extract_conversation(conv,i,data,comp_name)
        return conv,comp_name

In [8]:
from tqdm import tqdm


all_conversations = []
all_company_names = []
user_based_convs = []
for userid in tqdm(unique_users):
    user_conversations = []
    user_requests = data[(data["author_id"] == userid) & (data["in_response_to_tweet_id"]==-1)]
    for i in range(len(user_requests)):
        conv = "Customer: " + user_requests.iloc[i]["text"]
        responses = extract_responses(user_requests.iloc[i]["response_tweet_id"])
        for response in responses:
            convers,comp_name = extract_conversation(conv,response,data,comp_name=None)
            all_conversations.append(convers)
            all_company_names.append(comp_name)
            user_conversations.append(convers)

    user_based_convs.append(user_conversations)

100%|██████████| 10000/10000 [16:47<00:00,  9.93it/s] 


In [9]:
user_id_list = []

for i in range(len(user_based_convs)):
    user_id_list += [unique_users[i]] * len(user_based_convs[i])
    

df = pd.DataFrame({"user_id":user_id_list,"conversations":all_conversations,"company_name":all_company_names})
    

In [10]:
df

,user_id,conversations,company_name
0,115712,Customer: @sprintcare is the worst customer se...,sprintcare
1,115712,Customer: @sprintcare is the worst customer se...,sprintcare
2,115712,Customer: @sprintcare is the worst customer se...,sprintcare
3,115712,Customer: @115714 @sprintcare how can I get in...,sprintcare
4,115712,Customer: @115714 @sprintcare how can I get in...,sprintcare
...,...,...,...
33811,126216,"Customer: @GWRHelp no info, full platform - no...",GWRHelp
33812,126216,Customer: @GWRHelp I'm on one of your trains t...,GWRHelp
33813,126216,Customer: I feel for you @GWRHelp social media...,GWRHelp
33814,126216,Customer: @GWRHelp another evening of disrupti...,GWRHelp


In [11]:
def print_conv_History(user_id):
    conversations = df[df["user_id"] == user_id]["conversations"].values
    for i in conversations:
        print(i)
        print("\n")

print_conv_History("115767")

Customer: Any help here @AdobeCare? https://t.co/x50e57UG4E
Company: @115767 Hi Jason, please let us know if there is anything we can do to help. Thanks! ^AJ https://t.co/iuwZCjz4Or
Customer: @AdobeCare Pretty much explained my issue in the quoted tweet... dragging an image onto a canvas no longer center snaps it...
Company: @115767 Hi Jason, could you please try resetting the preferences of Photoshop &amp; let us know if it helps with the issue https://t.co/j3Dj2HDknM   ^SC
Customer: @AdobeCare the ctrl+alt+shift on load method didn't work. (2/2)
Company: @115767 Sorry that this isn't more simple... let us know if the steps in the video work out or if you need further help ^Madison
Customer: @AdobeCare Didn't work. Tried it on another computer, fresh install, same thing...
Company: @115767 Would you please DM the Adobe Product &amp; your purchase details that you have so that we can ask our experts to follow up. ^Raj https://t.co/iuwZCjz4Or
Customer: @AdobeCare sure thing
Customer: @A

In [13]:
df.to_excel("..\\Data/processed_conversations_10k.xlsx",index=False)